In [1]:
import os
SPARK_HOME = '/Users/isang-geon/spark'
# exec(open(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py')).read())
exec(open(os.path.join(SPARK_HOME, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.0.1
      /_/

Using Python version 3.8.5 (default, Sep  4 2020 02:22:02)
SparkSession available as 'spark'.


In [2]:
stations = sc.textFile(SPARK_HOME + '/data/bike-share/stations')

In [4]:
stations.take(5)

['2,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013',
 '3,San Jose Civic Center,37.330698,-121.888979,15,San Jose,8/5/2013',
 '4,Santa Clara at Almaden,37.333988,-121.894902,11,San Jose,8/6/2013',
 '5,Adobe on Almaden,37.331415,-121.8932,19,San Jose,8/5/2013',
 '6,San Pedro Square,37.336721,-121.894074,15,San Jose,8/7/2013']

In [5]:
status = sc.textFile(SPARK_HOME + '/data/bike-share/status')

In [6]:
status.take(5)

['10,9,6,"2015-02-28 23:59:01"',
 '10,9,6,"2015-02-28 23:58:02"',
 '10,9,6,"2015-02-28 23:57:02"',
 '10,8,7,"2015-02-28 23:56:02"',
 '10,8,7,"2015-02-28 23:55:02"']

In [7]:
status2 = status.map(lambda x: x.split(',')) \
                .map(lambda x: (x[0], x[1], x[2], x[3].replace('"',''))) \
                .map(lambda x: (x[0], x[1], x[2], x[3].split(' '))) \
                .map(lambda x: (x[0], x[1], x[2], x[3][0].split('-'), x[3][1].split(':'))) \
                .map(lambda x: (int(x[0]), int(x[1]), int(x[3][0]), int(x[3][1]), int(x[3][2]), int(x[4][0])))

In [8]:
status2.first()

(10, 9, 2015, 2, 28, 23)

In [9]:
status3 = status2.filter(lambda x: x[2]==2015 and \
                        x[3]==2 and \
                        x[4]>=22) \
                        .map(lambda x:(x[0], x[1], x[5]))

In [11]:
stations2 = stations.map(lambda x: x.split(',')) \
                    .filter(lambda x: x[5] == 'San Jose') \
                    .map(lambda x:(int(x[0]), x[1]))

In [12]:
stations2.first()

(2, 'San Jose Diridon Caltrain Station')

In [13]:
status_kv = status3.keyBy(lambda x: x[0])
stations_kv = stations2.keyBy(lambda x: x[0])

In [15]:
status_kv.first()

(10, (10, 9, 23))

In [16]:
stations_kv.first()

(2, (2, 'San Jose Diridon Caltrain Station'))

In [17]:
joined = status_kv.join(stations_kv)

In [18]:
joined.first()

(12, ((12, 10, 23), (12, 'SJSU 4th at San Carlos')))

In [19]:
cleaned = joined.map(lambda x:(x[0], x[1][0][1], x[1][0][2], x[1][1][1]))

In [20]:
cleaned.first()

(12, 10, 23, 'SJSU 4th at San Carlos')

In [21]:
avgbyhour = cleaned.keyBy(lambda x:(x[3],x[2])) \
                    .mapValues(lambda x:(x[1], 1)) \
                    .reduceByKey(lambda x, y:(x[0] + y[0], x[1] + y[1])) \
                    .mapValues(lambda x:(x[0]/x[1]))

In [22]:
avgbyhour.first()

(('SJSU 4th at San Carlos', 21), 11.507142857142858)

In [24]:
topavail = avgbyhour.keyBy(lambda x: x[1]) \
                    .sortByKey(ascending=False) \
                    .map(lambda x:(x[1][0][0], x[1][0][1], x[0]))
topavail.take(10)

[('San Jose Diridon Caltrain Station', 17, 16.590476190476192),
 ('San Jose Diridon Caltrain Station', 7, 16.492857142857144),
 ('San Jose Diridon Caltrain Station', 6, 16.34285714285714),
 ('San Jose Diridon Caltrain Station', 18, 16.21904761904762),
 ('San Jose Diridon Caltrain Station', 19, 15.64047619047619),
 ('San Jose Diridon Caltrain Station', 22, 15.516666666666667),
 ('San Jose Diridon Caltrain Station', 0, 15.445238095238095),
 ('San Jose Diridon Caltrain Station', 20, 15.416666666666666),
 ('San Jose Diridon Caltrain Station', 1, 15.392857142857142),
 ('San Jose Diridon Caltrain Station', 4, 15.383333333333333)]